In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
from os.path import join

import pandas as pd
import numpy as np

import missingno as msno

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt
import seaborn as sns

base_path = os.getcwd()+ '/FutureSales/data/competitive-data-science-predict-future-sales'

In [4]:
def prepare_train_data():
    train_file_path= base_path+'/sales_train.csv'
    
    train = pd.read_csv(train_file_path)


    # Summing sales for each month and clipping into [0, 20]
    train = train.groupby(['shop_id', 'item_id','date_block_num'])['item_cnt_day'].sum()
    train = train.reset_index()
    train['item_cnt_day'] = train['item_cnt_day'].map(lambda x: max(0, min(20, x)))
    
   
    # add month and year
    train['month'] = train['date_block_num'].map(lambda x: (x % 12) + 1)
    train['year'] = train['date_block_num'].map(lambda x: 2013 + x // 12)

    
    # Dropping sales in Dec.
    train = train[(train['month'] != 1)]
    
    return train


def prepare_test_data():
    test_file_path = base_path+'/test.csv'
    test = pd.read_csv(test_file_path)

    test['date_block_num'] = 34 
    test['month'] = 11
    test['year'] = 2015
    
    return test

In [5]:
train = prepare_train_data()
test = prepare_test_data()